In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('apartment_price_in_Kokshetau_kaggle.csv')
df.drop(columns=['Unnamed: 0', 
                 'is_in_kokshetau', 'street', 'latitude', 'longitude', 'distance_to_center', 'repair_status', 'type_of_house', 'distance_to_nearest_school', 'distance_to_nearest_kindergarten', 'distance_to_nearest_pharmacy'], axis=1, inplace=True)
df['area'] = df['area'].str.split(',').str[0]
df['area'] = df['area'].astype(float)
df = df[df['year'] != 2025]

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5122 entries, 0 to 5270
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   price           5122 non-null   float64
 1   area            5122 non-null   float64
 2   flat_toilets    5122 non-null   object 
 3   balcony         5122 non-null   object 
 4   current_floors  5122 non-null   float64
 5   total_floors    5122 non-null   float64
 6   ceiling         5122 non-null   float64
 7   dorm            5122 non-null   object 
 8   mortgage        5122 non-null   object 
 9   year            5122 non-null   int64  
dtypes: float64(5), int64(1), object(4)
memory usage: 440.2+ KB


In [3]:
top_10_price = df.sort_values(by='price', ascending=False).head(10)
top_10_area = df.sort_values(by='area', ascending=False).head(10)
top_10_ceiling = df.sort_values(by='ceiling', ascending=False).head(10)
print(top_10_price)
print(top_10_area)
print(top_10_ceiling)

            price   area   flat_toilets balcony  current_floors  total_floors  \
2518  185000000.0   62.6     раздельный     нет             1.0           5.0   
86    180800000.0  272.0     раздельный     нет             9.0          10.0   
1909  105000000.0  142.9     раздельный     нет             7.0           9.0   
18    101601245.0  141.5     раздельный     нет             9.0          10.0   
702   100000000.0  151.0     раздельный     нет             3.0           9.0   
211   100000000.0  141.0     раздельный     нет             2.0           9.0   
799   100000000.0  141.8     раздельный     нет             5.0           9.0   
648   100000000.0  156.0  2 с/у и более     нет            10.0          13.0   
638   100000000.0  141.0  2 с/у и более     нет             2.0           9.0   
4175  100000000.0  151.0     раздельный     нет             3.0           9.0   

      ceiling dorm mortgage  year  
2518      2.7  нет      нет  1992  
86        3.4  нет      нет  2020  


In [4]:
df = df.drop(index=[2518, 582, 4870, 1309, 3051, 559, 4624, 3833, 1828, 852])
df.reset_index(drop=True, inplace=True)

In [5]:
df['balcony'] = df['balcony'].apply(lambda x: 0 if x == 'нет' else 1)
df['flat_toilets'] = df['flat_toilets'].apply(lambda x: 0 if x == 'нет' else 1)

df['mortgage'] = df['mortgage'].map({'нет': 0, 'В залоге': 1})
df['dorm'] = df['dorm'].map({'нет': 0, 'да': 1})

print(df.isnull().sum())

price             0
area              0
flat_toilets      0
balcony           0
current_floors    0
total_floors      0
ceiling           0
dorm              0
mortgage          0
year              0
dtype: int64


In [6]:
#Create features 
df["age_of_building"] = 2025 - df["year"]  
df["floor_ratio"] = df["current_floors"] / df["total_floors"]  

In [7]:
=from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

X = df.drop(columns=['price'])  
y = df['price']  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestRegressor(max_depth=30, max_features= 'sqrt', min_samples_leaf= 1, min_samples_split=2, n_estimators= 300)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(r2_score(y_test, y_pred))
print(mean_absolute_error(y_test, y_pred))
print(mean_squared_error(y_test, y_pred))

SyntaxError: invalid syntax (1905453866.py, line 1)

In [ ]:
import pickle
with open("rf_model.pkl", "wb") as f:
    pickle.dump(model, f)
